In [16]:
import gradio as gr
from PIL import Image
import torch
import torchvision.transforms as T
import torchvision.models as models
from torchvision.models import resnet18, ResNet18_Weights

#  Pretrained model (ResNet18, ImageNet)
weights = ResNet18_Weights.DEFAULT  # proper weights enum
model = resnet18(weights=weights)
model.eval()

#  Image transform (from weights metadata)
transform = weights.transforms()

#  Get ImageNet class labels from weights metadata
labels = weights.meta["categories"]

# Predict function: input img -> output dict
def predict(img):
    img = img.convert("RGB")
    x = transform(img).unsqueeze(0)
    with torch.no_grad():
        y = model(x)
        probs = torch.softmax(y[0], dim=0)
    top_p, top_i = torch.topk(probs, 3)
    return {labels[top_i[j]]: float(top_p[j]) for j in range(3)}

#  Gradio interface
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Label(num_top_classes=3, label="Prediction"),
    title="Animal Image Classification",
    description="Upload an image and see the top 3 ImageNet predictions."
)

demo.launch()



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://87510b4326be1b1599.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [17]:
# ==========================
#  Install required packages
# ==========================
pip install gradio torch torchvision transformers opencv-python-headless --quiet

# ==========================
#  Imports
# ==========================
import gradio as gr
import torch
import torchvision.transforms as T
import torchvision.models as models
from torchvision.models import ResNet18_Weights
from PIL import Image
import numpy as np
import cv2
from transformers import pipeline

# ==========================
#  Load Image Classification Model (ResNet18)
# ==========================
weights = ResNet18_Weights.DEFAULT
model = models.resnet18(weights=weights)
model.eval()

# ImageNet labels (built-in, no file needed)
CLASS_NAMES = weights.meta["categories"]

# Transform
transform = weights.transforms()

# ==========================
#  Load LLM (Text Generation)
# ==========================
llm = pipeline(
    "text-generation",
    model="distilgpt2",
    max_new_tokens=50
)

# ==========================
#  Grad-CAM Implementation (XAI)
# ==========================
def gradcam(img_tensor, model, target_layer="layer4"):
    gradients = []
    activations = []

    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0])

    def forward_hook(module, input, output):
        activations.append(output)

    layer = dict(model.named_modules())[target_layer]
    layer.register_forward_hook(forward_hook)
    layer.register_backward_hook(backward_hook)

    output = model(img_tensor)
    pred_class = output.argmax()
    output[0, pred_class].backward()

    grads = gradients[0]
    acts = activations[0]

    weights = grads.mean(dim=(2, 3), keepdim=True)
    cam = (weights * acts).sum(dim=1).squeeze()
    cam = cam.detach().numpy()
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()

    cam = cv2.resize(cam, (224, 224))
    return cam

# ==========================
#  Prediction + XAI + LLM
# ==========================
def predict(img):
    img = img.convert("RGB")
    img_tensor = transform(img).unsqueeze(0)
    img_tensor.requires_grad = True

    # Prediction
    with torch.no_grad():
        logits = model(img_tensor)
        probs = torch.softmax(logits[0], dim=0)

    top_p, top_i = torch.topk(probs, 3)

    # Grad-CAM
    cam = gradcam(img_tensor, model)
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    img_np = np.array(img.resize((224, 224)))
    overlay = cv2.addWeighted(img_np, 0.6, heatmap, 0.4, 0)

    # LLM Explanation
    label = CLASS_NAMES[top_i[0]]
    prompt = f"Explain in simple words why an image might be classified as {label}."
    explanation = llm(prompt)[0]["generated_text"]

    return (
        {CLASS_NAMES[top_i[i]]: float(top_p[i]) for i in range(3)},
        overlay,
        explanation
    )

# ==========================
# Gradio GUI
# ==========================
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=[
        gr.Label(num_top_classes=3, label="Prediction"),
        gr.Image(label="Grad-CAM (XAI)"),
        gr.Textbox(label="LLM Explanation")
    ],
    title="🧠 Image Classification with GUI + XAI + LLM",
    description="Upload an image → get prediction, explainable AI heatmap, and LLM-based reasoning."
)

demo.launch()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eef0bc431f438a0376.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
